In [160]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

In [161]:
# The probability margin is the absolute value of the difference
# between the two teams in terms of who is more likely to win: np.abs(df['prob1'] - df['prob2']).
# There is also probtie which predicts the probability the game ends up level.
# Simple three-layer neural network to anticipate which
# bucket of probability margin a given match falls into:
    # If X is less than the mean probability margin (MPM), it's "WITHIN MEAN PROBABILITY MARGIN."
    # If X is greater than or equal to MPM, but less than MPM + 1 standard deviation (SDPM), it's "SLIGHT PROBABILITY ADVANTAGE."
    # If X is greater than MPM + SDPM, but less than MPM + (SDPM * 1.5), it's "MODERATE PROBABILITY ADVANTAGE."
    # If X is greater than MPM + (SDPM * 1.5), meaning it is an "outlier", it's "SIGNIFICANT PROBABILITY ADVANTAGE."
# We will use statistics from the Barclays Premier League from the start of the 2018-19 season.

df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv")
df = df[(df.league=="Barclays Premier League") & (df.season >= 2018)]

In [162]:
# A three-layer model is idea because it has enough filtering capability to generalize and get rid of the noise.
# That said, we will throw in an l2 regularizer just to trim it even further.
def load_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(8, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [163]:
# The y variables need to be converted from their existing state after train_test_split to categorical.
# That said, it's a bit more than just calling to_categorical...the array in question has to be factorized, then converted
# to a NumPy array, then converted to categorical.
# This results in a four-column array of one-hot values for the variables.
# We have to do it twice, so best to create a basic function for it.
def cat(array):
    array = tf.keras.utils.to_categorical(np.asarray(pd.factorize(array)[0]))
    return array

In [164]:
# OK, now the fun part. Probability margin, MPM, and SDPM generation.
df['prob_margin'] = np.abs(df['prob1'] - df['prob2'])
mpm = np.mean(df['prob_margin'])
sdpm = np.std(df['prob_margin'])

In [165]:
# Empty list.
pm = []

In [166]:
# It's bucketing time.
for x in df['prob_margin']:
    if x < mpm:
        pm.append("WITHIN MEAN PROBABILITY MARGIN")
    elif (x >= mpm) & (x < (mpm + sdpm)):
        pm.append("SLIGHT PROBABILITY ADVANTAGE")
    elif (x >= (mpm + sdpm)) & (x < (mpm + (sdpm * 1.5))):
        pm.append("MODERATE PROBABILITY ADVANTAGE")
    else:
        pm.append("SIGNIFICANT PROBABILITY ADVANTAGE")

In [167]:
# Assigning PM to our new column, prob_bucket.
df['prob_bucket'] = pm

In [168]:
# Since there are four output classes, we need to have the identical number of columns as our predictor variables (4).
# We haven't used probtie yet, but it would be good to include.
X_final = df[['prob1', 'prob2', 'prob_margin', 'probtie']]
y_final = df[['prob_bucket']].copy()
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=6)

In [169]:
# Converting y_train and y_test to categorical.
y_train = cat(y_train["prob_bucket"])
y_test = cat(y_test["prob_bucket"])

In [170]:
# We want to see if there is a more optimal number of epochs.
# We will set it to tell us with a patience setting of 5.
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True)

In [171]:
# Best to generate new train and test samples to see that our results are reproducible.
# First thing to do is shuffle the X and y dataframes with the sample function.
X_final_sampled = X_final.sample(frac=1)
y_final_sampled = y_final.sample(frac=1)
X_train, X_test, y_train, y_test = train_test_split(X_final_sampled, y_final_sampled, test_size=0.2, random_state=6)

In [172]:
# Second time around categorization.
y_train = cat(y_train['prob_bucket'])
y_test = cat(y_test['prob_bucket'])

In [173]:
# Optimizing the model based on the resampled data.
model_opt = load_model()
history = model_opt.fit(X_train, y_train, batch_size=128, epochs=25, validation_data=(X_test, y_test), callbacks=[early_stop])

Epoch 1/25
8/8 [==============================] - 0s 16ms/step - loss: 1.3862 - accuracy: 0.3092 - val_loss: 1.3559 - val_accuracy: 0.1886
Epoch 2/25
8/8 [==============================] - 0s 2ms/step - loss: 1.3701 - accuracy: 0.5548 - val_loss: 1.3482 - val_accuracy: 0.1667
Epoch 3/25
8/8 [==============================] - 0s 2ms/step - loss: 1.3550 - accuracy: 0.5592 - val_loss: 1.3430 - val_accuracy: 0.1667
Epoch 4/25
8/8 [==============================] - 0s 2ms/step - loss: 1.3410 - accuracy: 0.5592 - val_loss: 1.3401 - val_accuracy: 0.1667
Epoch 5/25
8/8 [==============================] - 0s 2ms/step - loss: 1.3266 - accuracy: 0.5592 - val_loss: 1.3415 - val_accuracy: 0.1667
Epoch 6/25
8/8 [==============================] - 0s 2ms/step - loss: 1.3127 - accuracy: 0.5592 - val_loss: 1.3475 - val_accuracy: 0.1667
Epoch 7/25
8/8 [==============================] - 0s 2ms/step - loss: 1.2976 - accuracy: 0.5592 - val_loss: 1.3583 - val_accuracy: 0.1667
Epoch 8/25
8/8 [=================

In [174]:
# We could see the number of epochs, but easier to just tie it to an object.
num_epochs = len(history.__dict__['epoch'])

In [175]:
# Given that earlystopping stopped after 9 epochs, we will run 9 as well.
# Again, since it's tied to an object, not much extra work involved.
model.fit(X_train, y_train, epochs=num_epochs, batch_size=128, verbose=1)

Epoch 1/9
8/8 [==============================] - 0s 572us/step - loss: 1.1356 - accuracy: 0.5592
Epoch 2/9
8/8 [==============================] - 0s 572us/step - loss: 1.1337 - accuracy: 0.5592
Epoch 3/9
8/8 [==============================] - 0s 572us/step - loss: 1.1320 - accuracy: 0.5592
Epoch 4/9
8/8 [==============================] - 0s 572us/step - loss: 1.1309 - accuracy: 0.5592
Epoch 5/9
8/8 [==============================] - 0s 429us/step - loss: 1.1297 - accuracy: 0.5592
Epoch 6/9
8/8 [==============================] - 0s 572us/step - loss: 1.1291 - accuracy: 0.5592
Epoch 7/9
8/8 [==============================] - 0s 572us/step - loss: 1.1282 - accuracy: 0.5592
Epoch 8/9
8/8 [==============================] - 0s 572us/step - loss: 1.1278 - accuracy: 0.5592
Epoch 9/9
8/8 [==============================] - 0s 572us/step - loss: 1.1275 - accuracy: 0.5592


In [176]:
# Generating the predictions.
predictions = model.predict(X_test)

In [177]:
# We'll take it.
loss, accuracy = model.evaluate(y_test, predictions)

8/8 [==============================] - 0s 715us/step - loss: 1.1523 - accuracy: 1.0000


In [178]:
# For good measure to close it out.
print("Test Accuracy: {}%".format(accuracy * 100))

Test Accuracy: 100.0%
